## Download dados de chuva

In [1]:
from pathlib import Path
import os

caminho_base = os.getcwd()

diretorio_saida = Path(f'{caminho_base}/downloads')
diretorio_saida.mkdir(exist_ok=True, parents=True)

#### Observado Satélite - MERGE

In [2]:
from src import cptec_merge

In [3]:
# COMO USAR ESSA BIBLIO : https://github.com/Ramonmurilo/MERGE_CPTEC
nome_arquivo_merge = cptec_merge.baixar_dados(discretizacao='diario',hoje=True, diretorio_saida=diretorio_saida)

MERGE_CPTEC_20210930.grib2 [ok]


#### Previsto CFSv2

In [4]:
from src import cfs
import pendulum

In [5]:
nome_arquivo_cfsv = cfs.baixa_mensal(hoje=True, output=diretorio_saida)

SepIC_cfsv2_precip_anom_stdanom.ctl [ok]
SepIC_cfsv2_precip_anom_stdanom.dat [ok]
/mnt/c/scripts/smapy/downloads/SepIC_cfsv2_precip_anom_stdanom.nc [ok]


## Recorte

In [6]:
#import requests
#from pathlib import Path
#import pendulum
#import sys
#import os
import geopandas as gpd

In [7]:
contornos = gpd.read_file(f"{caminho_base}/contornos/contornos_smap.shp")

In [8]:
contornos

,Nome,FID,geometry
0,Barra Bonita-Promissão,NaN,"POLYGON ((-49.51100 -20.81469, -49.50097 -20.8..."
1,São Simão,NaN,"POLYGON ((-47.48836 -15.45417, -47.48396 -15.4..."
2,Três Marias,NaN,"MULTIPOLYGON (((-45.81222 -20.46639, -45.81188..."
3,Itá,NaN,"MULTIPOLYGON (((-50.74444 -28.62556, -50.74454..."
4,Foz do Areia,NaN,"MULTIPOLYGON (((-50.84889 -26.74778, -50.84898..."
5,Serra da Mesa,NaN,"MULTIPOLYGON (((-49.01139 -16.14137, -49.01106..."
6,Capivara,NaN,"MULTIPOLYGON (((-49.96473 -25.49971, -49.96482..."
7,Furnas,NaN,"MULTIPOLYGON (((-45.47389 -22.71944, -45.47355..."
8,Água Vermelha,NaN,"MULTIPOLYGON (((-45.47389 -22.71944, -45.47355..."
9,Emborcação,NaN,"MULTIPOLYGON (((-47.75861 -18.80777, -47.75871..."


#### Recorte previsão | CFSv2

In [9]:
from src import recorte_chuva
import xarray as xr

In [ ]:
dados = xr.open_dataset(f'{nome_arquivo_cfsv}')
dado_convertido = dados.precip*30

In [12]:
print('nome bacia : [lead1 lead2 ... lead6]\n')
for shapefile in range(0,len(contornos)):
    
    # Emborcação tá com problema | to gerando Rosana 2x
    shapefile = 10 if shapefile==9 else shapefile # A chuva do MERGE é recortada para somente o brasil, se a bacia estiver fora do território dá problema ('-.-)
    
    dado_recortado = recorte_chuva.main(dados=dado_convertido, contorno=contornos.geometry[shapefile], tipo='nc')
    media_bacia = dado_recortado.mean(dim=["lat", "lon"])
    print(f'{contornos.Nome[shapefile]} : {media_bacia.values}')

nome bacia : [lead1 lead2 ... lead6]

Barra Bonita-Promissão : [112.051346 106.778244 153.44235  184.57068  167.1435   103.93921 ]
São Simão : [114.75792 179.10951 246.51021 276.15622 264.47482 220.34431]
Três Marias : [131.5413  191.36519 250.04158 258.7248  221.71861 152.19899]
Itá : [206.23816 140.25555 163.64632 177.66743 159.00096 134.56935]
Foz do Areia : [185.97601 145.3329  152.8551  170.06761 158.41711 125.9253 ]
Serra da Mesa : [126.64552 231.68631 300.14752 307.35858 315.64484 297.66486]
Capivara : [150.39069 125.67454 151.92157 197.72911 179.08694 122.45074]
Furnas : [180.33192 201.92813 253.96416 281.06512 260.61264 161.82341]
Água Vermelha : [145.31468 165.17589 223.57619 254.82532 237.48528 156.89362]
Rosana : [147.4362  122.8023  150.2658  195.5466  177.12991 121.3479 ]
Rosana : [147.4362  122.8023  150.2658  195.5466  177.12991 121.3479 ]
Promissão : [127.97999 120.7791  161.87851 202.34006 183.99757 112.97272]
Tocantins : [ 60.99182 182.35307 284.87085 306.15152 336.1

#### Recorte observado | MERGE

In [13]:
from src import recorte_chuva
import xarray as xr

In [14]:
dados = xr.open_dataset(f'{diretorio_saida}/{nome_arquivo_merge}')
dado_convertido = dados.prec

In [48]:
print('nome bacia : observado do dia baixado \n')
for shapefile in range(0,len(contornos)):
    
    dado_recortado = recorte_chuva.main(dados=dado_convertido, contorno=contornos.geometry[shapefile], tipo='grib')
    media_bacia = dado_recortado.mean(dim=["latitude", "longitude"])
    print(f'{contornos.Nome[shapefile]} : {media_bacia.values}')

nome bacia : observado do dia baixado 

Barra Bonita-Promissão : 0.24302326142787933
São Simão : 0.445016473531723
Três Marias : 2.6150341033935547
Itá : 0.9672029614448547
Foz do Areia : 0.08535447716712952
Serra da Mesa : 1.774369239807129
Capivara : 0.23290172219276428
Furnas : 2.7045199871063232
Água Vermelha : 1.2514450550079346
Emborcação : 0.3428714871406555
Rosana : 0.2178051471710205
Promissão : 0.1242469847202301
Tocantins : 0.3932192623615265
Itaipu : 1.1688050031661987
Sobradinho : 0.6278287172317505
Barra Bonita : 0.03496503457427025
Iguaçu : 0.8700248599052429
